# Prepare Sources

The HMRC provide overseas trade statistics broken down by country and commoditiy code using the Combined Nomenclature "CN8" 8 digit codes.

These statistics have been obtained as a series of CSV files as "Tidy Data".

However, some preparation is necessary in order to process these files using the table2qb utility.

Firstly, fetch the source data, in this case from a shared (open) Google drive.

We also keep track of the processing and the provenance of the inputs and outputs using W3C Prov.

In [1]:
from datetime import datetime
import json
from pytz import timezone
from os import environ

provActivity = {
    '@id': environ.get('BUILD_URL', 'unknown-build') + "#prepare_sources",
    '@type': 'activity',
    'startedAtTime': datetime.now(timezone('Europe/London')).isoformat(),
    'label': 'Prepare sources',
    'comment': 'Jupyter Python notebook as part of Jenkins job %s' % environ.get('JOB_NAME', 'unknown-job')
}

In [2]:
import requests
from pathlib import Path
from io import BytesIO
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

provSources = []

sources = [
    ('CN8_Non-EU_cod_2012.csv', '1P7YyFF6qXKXWVtR0Vt3kkvFPOjThMQH8'),
    ('CN8_Non-EU_cod_2013.csv', '1de-Le9ungrbdoGyvWI_RwmEhNpTmR-70'),
    ('CN8_Non-EU_cod_2014.csv', '1oC3jlItfsUshd54KOR7yn9NxpR83iCbC'),
    ('CN8_Non-EU_cod_2015.csv', '1H54-FYrCFa1DylCBg38RAPAeCtkGq4la'),
    ('CN8_Non-EU_cod_2016.csv', '11fLsnoiWzTcA1d3nSDWvyrKQEHwIf6Hz')
]

sourceUrls = []

for filename, google_id in sources:
    
    sourceUrl = f'https://drive.google.com/uc?export=download&id={google_id}'
    sourceUrls.append(sourceUrl)
    provSources.append({
        '@id': sourceUrl,
        '@type': 'entity',
        'label': filename,
        'wasUsedBy': provActivity['@id']
    })

In [3]:
import pandas as pd

table = pd.concat([pd.read_csv(BytesIO(session.get(sourceUrl).content),
                                       dtype={'comcode': str},
                                       na_values=[], keep_default_na=False)
                       for sourceUrl in sourceUrls], ignore_index=True).rename(
    index = str,
    columns = {'year': 'Year', 'flow': 'Flow', 'comcode': 'Commodity',
               'country': 'Foreign Country', 'svalue': 'Value'})
table

,Year,Flow,Commodity,Foreign Country,Value
0,2012,e,01012100,Norway,1773490
1,2012,e,01012100,Switzerland,69378
2,2012,e,01012100,Turkey,406337
3,2012,e,01012100,Ukraine,49903
4,2012,e,01012100,Serbia,32550
5,2012,e,01012100,Libya,152125
6,2012,e,01012100,Nigeria,2578
7,2012,e,01012100,South Africa,1390218
8,2012,e,01012100,United States,16701807
9,2012,e,01012100,Canada,30459


Countries are mandated by Eurostat to use the Geonomenclature (GEONOM), which gradually changes over the years. HMRC keeps track of these changes to the country codes and numbers and their data for each year will use the latest GEONOM codes.

For now, we'll just use a static list that's good enough, but will __need to revisit this__.

In [4]:
geonom_2018_excel = 'https://drive.google.com/uc?export=download&id=17Laouuze9gT04xV1Q5M-RZyEqGZUHZJ_'
geonom = pd.read_excel(BytesIO(session.get(geonom_2018_excel).content),
                       na_values=[], keep_default_na=False, dtypes=str)
geonom['codseq'] = geonom['codseq'].apply(
    lambda x: "%03d" % int(x))
geonom.drop(columns=['statsw', 'geogsw', 'dutysw'], inplace=True)
geonom

,codseq,country,codalpha
0,001,France,FR
1,003,Netherlands,NL
2,004,Germany,DE
3,005,Italy,IT
4,006,UK,GB
5,007,Irish Republic,IE
6,008,Denmark,DK
7,009,Greece,GR
8,010,Portugal,PT
9,011,Spain,ES


We'll ignore the miscellaneous codes (e.g. Stores & Provis: deliveries of ship/aircraft stores et seq.)

In [5]:
geonom = geonom[:geonom[geonom['country'] == 'Stores & Provis.'].index[0]]
geonom.tail()

,codseq,country,codalpha
232,891,Antarctica,AQ
233,892,Bouvet Island,BV
234,894,Fr Southern Terr,TF
235,923,Niue Island,NU
236,926,Cook Islands,CK


In [6]:
table = pd.merge(table, geonom, how='inner', left_on='Foreign Country', right_on='country', validate="m:1")
table

,Year,Flow,Commodity,Foreign Country,Value,codseq,country,codalpha
0,2012,e,01012100,Norway,1773490,028,Norway,NO
1,2012,e,01012990,Norway,147704,028,Norway,NO
2,2012,e,01051200,Norway,161586,028,Norway,NO
3,2012,e,01051300,Norway,11096,028,Norway,NO
4,2012,e,01061900,Norway,4520,028,Norway,NO
5,2012,e,01069000,Norway,17972,028,Norway,NO
6,2012,e,02011000,Norway,183545,028,Norway,NO
7,2012,e,02012090,Norway,278155,028,Norway,NO
8,2012,e,02013000,Norway,4287,028,Norway,NO
9,2012,e,02023090,Norway,4907,028,Norway,NO


We're using a good-enough-for-now list of country codes based on the alpha codes above

In [7]:
table.rename(columns={'codalpha': 'HMRC Partner Geography'}, inplace=True)
table.drop(columns=['Foreign Country', 'codseq', 'country'], inplace=True)
table

,Year,Flow,Commodity,Value,HMRC Partner Geography
0,2012,e,01012100,1773490,NO
1,2012,e,01012990,147704,NO
2,2012,e,01051200,161586,NO
3,2012,e,01051300,11096,NO
4,2012,e,01061900,4520,NO
5,2012,e,01069000,17972,NO
6,2012,e,02011000,183545,NO
7,2012,e,02012090,278155,NO
8,2012,e,02013000,4287,NO
9,2012,e,02023090,4907,NO


In [8]:
table['Measure Type'] = 'GBP Total'
table['Unit'] = '£ million'
table['Flow'] = table['Flow'].map(lambda x: {'i': 'Import', 'e': 'Export'}[x])

In [9]:
destFolder = Path('out')
destFolder.mkdir(exist_ok=True, parents=True)

provOutputs = []

destFile = destFolder / 'CN8_Non-EU_cod_2012-2016.csv'
table.sample(n=10000, random_state=149).to_csv(destFile, index=False)
#table.to_csv(destFile, index=False)
provOutputs.append((destFile, 'CN8_Non-EU_cod-2012-2016 table'))

Output the PROV metadata as JSON-LD. This goes to the 'out' folder.

In [10]:
metadataDir = Path('metadata')
with open(metadataDir / 'prov_context.json') as contextFile:
    context = json.load(contextFile)

provActivity['endedAtTime'] = datetime.now(timezone('Europe/London')).isoformat()
prov = {
    '@context': context,
    '@graph': [ provActivity ] + provSources + [
        {
            '@id': environ.get('BUILD_URL', 'unknown-build') + 'artifact/' + str(filename),
            '@type': 'entity',
            'wasGeneratedBy': provActivity['@id'],
            'label': label
        } for (filename, label) in provOutputs
    ]
}

with open(destFolder / 'prov.jsonld', 'w') as provFile:
    json.dump(prov, provFile, indent=2)

Create dataset metadata

In [11]:
modified_date = datetime.now(timezone('Europe/London')).isoformat()

from string import Template
with open(Path('metadata') / 'dataset.trig.template', 'r') as metadata_template_file:
    metadata_template = Template(metadata_template_file.read())
    with open(destFolder / 'dataset.trig', 'w') as metadata_file:
        metadata_file.write(metadata_template.substitute(modified=modified_date))